In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout,Activation

from keras.layers.convolutional import Conv1D
from keras import backend as K
from keras.layers.core import Lambda
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding

import keras
from keras.utils import np_utils

In [5]:
data = pd.read_csv("IDS_Combined_Data_IoT2.csv")

In [6]:
data.head(5)

,ts,date,time,fridge_temperature,temp_condition,label,type,door_state,sphone_signal,latitude,...,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,motion_status,light_status,current_temperature,thermostat_status,temperature,pressure,humidity
0,1556245180,25-Apr-19,19:19:40,9.00,1,1,ddos,0,0,4.514077,...,32708,32035,32728,0,0,28.442693,1,23.016184,1.035,46.343618
1,1556245180,25-Apr-19,19:19:40,9.25,1,1,ddos,0,0,4.514077,...,32708,32035,32728,0,0,28.442693,1,38.620990,1.035,46.343618
2,1556245185,25-Apr-19,19:19:45,12.65,1,1,ddos,0,0,4.514077,...,32708,32035,32728,0,0,28.442693,1,42.732741,1.035,46.343618
3,1556245185,25-Apr-19,19:19:45,4.65,0,1,ddos,0,0,4.514077,...,32708,32035,32728,0,0,28.442693,1,37.785562,1.035,46.343618
4,1556245195,25-Apr-19,19:19:55,12.65,1,1,ddos,0,0,4.514077,...,32708,32035,32728,0,0,28.442693,1,36.353584,1.035,46.343618


In [7]:
dataset = data.drop(['ts', 'date', 'time'], axis =1)

In [8]:
dataset.head(5)

,fridge_temperature,temp_condition,label,type,door_state,sphone_signal,latitude,longitude,FC1_Read_Input_Register,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,motion_status,light_status,current_temperature,thermostat_status,temperature,pressure,humidity
0,9.00,1,1,ddos,0,0,4.514077,14.421946,32450,32708,32035,32728,0,0,28.442693,1,23.016184,1.035,46.343618
1,9.25,1,1,ddos,0,0,4.514077,14.421946,32450,32708,32035,32728,0,0,28.442693,1,38.620990,1.035,46.343618
2,12.65,1,1,ddos,0,0,4.514077,14.421946,32450,32708,32035,32728,0,0,28.442693,1,42.732741,1.035,46.343618
3,4.65,0,1,ddos,0,0,4.514077,14.421946,32450,32708,32035,32728,0,0,28.442693,1,37.785562,1.035,46.343618
4,12.65,1,1,ddos,0,0,4.514077,14.421946,32450,32708,32035,32728,0,0,28.442693,1,36.353584,1.035,46.343618


In [9]:
dataset['label'].value_counts()

0    245000
1    156119
Name: label, dtype: int64

In [10]:
dataset.isna().sum().sum()

0

In [11]:
dataset.columns

Index(['fridge_temperature', 'temp_condition', 'label', 'type', 'door_state',
       'sphone_signal', 'latitude', 'longitude', 'FC1_Read_Input_Register',
       'FC2_Read_Discrete_Value', 'FC3_Read_Holding_Register', 'FC4_Read_Coil',
       'motion_status', 'light_status', 'current_temperature',
       'thermostat_status', 'temperature', 'pressure', 'humidity'],
      dtype='object')

In [12]:
X = dataset.drop(['label','type'], axis = 1)
X.shape

(401119, 17)

In [13]:
y = dataset['label']
y.shape

(401119,)

In [14]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8))) #don't change these values
X_T = sel.fit(X)

In [15]:
sel_cols = X_T.get_support(indices=True)
X_VT = X.iloc[:,sel_cols]
X_VT

,fridge_temperature,latitude,longitude,FC1_Read_Input_Register,FC2_Read_Discrete_Value,FC3_Read_Holding_Register,FC4_Read_Coil,current_temperature,temperature,pressure,humidity
0,9.00,4.514077,14.421946,32450,32708,32035,32728,28.442693,23.016184,1.035000,46.343618
1,9.25,4.514077,14.421946,32450,32708,32035,32728,28.442693,38.620990,1.035000,46.343618
2,12.65,4.514077,14.421946,32450,32708,32035,32728,28.442693,42.732741,1.035000,46.343618
3,4.65,4.514077,14.421946,32450,32708,32035,32728,28.442693,37.785562,1.035000,46.343618
4,12.65,4.514077,14.421946,32450,32708,32035,32728,28.442693,36.353584,1.035000,46.343618
...,...,...,...,...,...,...,...,...,...,...,...
401114,6.70,4.514077,14.421946,32450,32708,32035,32728,28.442693,24.250404,2.204924,37.024913
401115,6.70,4.514077,14.421946,32450,32708,32035,32728,28.442693,20.155984,-2.030547,90.297894
401116,6.70,4.514077,14.421946,32450,32708,32035,32728,28.442693,24.577566,0.872942,37.687701
401117,6.70,4.514077,14.421946,32450,32708,32035,32728,28.442693,20.913710,3.168207,93.647950


In [16]:
scaler = MinMaxScaler()
X_M = scaler.fit_transform(X_VT)
print(X_M)

[[0.61538462 0.00821665 0.00811167 ... 0.10772138 0.53355618 0.46251056]
 [0.63461538 0.00821665 0.00811167 ... 0.66504042 0.53355618 0.46251056]
 [0.89615385 0.00821665 0.00811167 ... 0.81188986 0.53355618 0.46251056]
 ...
 [0.43846154 0.00821665 0.00811167 ... 0.16348547 0.52695586 0.3756012 ]
 [0.43846154 0.00821665 0.00811167 ... 0.03263243 0.62043775 0.93746771]
 [0.43846154 0.00821665 0.00811167 ... 0.04525255 0.58120503 0.3689465 ]]


In [17]:
trainX, testX, trainY, testY = train_test_split(X_M, y, test_size = 0.20, random_state = 40)

In [18]:
print(X_M.shape)
print(y.shape)
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(401119, 11)
(401119,)
(320895, 11)
(320895,)
(80224, 11)
(80224,)


In [19]:
#Test Train split
lr = LogisticRegression() #for binary
lr.fit(trainX, trainY)
y_predict = lr.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

KeyboardInterrupt: 

In [ ]:
#Test Train split
lda = LinearDiscriminantAnalysis()
lda.fit(trainX, trainY)
y_predict = lda.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

In [ ]:
#Test Train split
#default parameters used in base paper (n_neighbors, default=5) (p, default=2 for Euclidean Distance)
knn = KNeighborsClassifier()
knn.fit(trainX, trainY)
y_predict = knn.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

In [ ]:
from sklearn.model_selection import GridSearchCV
#Grid search CV
model = KNeighborsClassifier()

parameters = [{'n_neighbors': [i for i in range(1, 21, 2)], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']}]

grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4, n_jobs = -1)

grid_search = grid_search.fit(trainX, trainY)

print('Best Results with Grid Search:')
print(grid_search.best_score_)
print(grid_search.best_params_)

y_pred = grid_search.predict(testX)

print('\nAccuracy Score  on test data: ' + str(accuracy_score(testY, y_pred)))
print(confusion_matrix(testY,y_pred))
print(classification_report(testY,y_pred))

In [61]:
#Test Train split
rf = RandomForestClassifier(n_estimators=10,criterion='gini')
rf.fit(trainX, trainY)
y_predict1 = rf.predict(testX)
print(accuracy_score(testY, y_predict1))
print(classification_report(testY, y_predict1))

0.8730180494615077
              precision    recall  f1-score   support

           0       0.88      0.92      0.90     48896
           1       0.86      0.81      0.83     31328

    accuracy                           0.87     80224
   macro avg       0.87      0.86      0.86     80224
weighted avg       0.87      0.87      0.87     80224



In [59]:
#Test Train Split
dt = DecisionTreeClassifier(criterion='gini')
dt.fit(trainX, trainY)
y_predict1 = dt.predict(testX)
print(accuracy_score(testY, y_predict1))
print(classification_report(testY, y_predict1))

0.8542830075787794
              precision    recall  f1-score   support

           0       0.88      0.88      0.88     48896
           1       0.81      0.81      0.81     31328

    accuracy                           0.85     80224
   macro avg       0.85      0.85      0.85     80224
weighted avg       0.85      0.85      0.85     80224



In [57]:
#Test Train Split
nb = GaussianNB()
nb.fit(trainX, trainY)
y_predict1 = nb.predict(testX)
print(accuracy_score(testY, y_predict1))
print(classification_report(testY, y_predict1))

0.7592491025129637
              precision    recall  f1-score   support

           0       0.99      0.61      0.76     48896
           1       0.62      0.99      0.76     31328

    accuracy                           0.76     80224
   macro avg       0.81      0.80      0.76     80224
weighted avg       0.85      0.76      0.76     80224



In [ ]:
#Test Train Split
svclassifier = SVC(kernel='rbf', gamma='auto')
svclassifier.fit(trainX, trainY)
y_predict = svclassifier.predict(testX)
print(accuracy_score(testY, y_predict))
print(classification_report(testY, y_predict))

### LSTM

In [62]:
X_M.shape

(401119, 11)

In [64]:
X1 = X_M.reshape((-1, 1, 11))

In [65]:
X1.shape

(401119, 1, 11)

In [66]:
(trainX, testX, trainY, testY) = train_test_split(X1, y, test_size = 0.20, random_state = 40)

In [67]:
# 1 LSTM Layer (input), 3 Dense Hidden Layers
model = Sequential()
model.add(LSTM(12, input_shape=(1, 11), activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(trainX, trainY, epochs=35, batch_size=64, verbose=2, validation_data=(testX, testY))

Epoch 1/35
5014/5014 - 22s - loss: 0.3692 - accuracy: 0.8706 - val_loss: 0.3511 - val_accuracy: 0.8780
Epoch 2/35
5014/5014 - 20s - loss: 0.3510 - accuracy: 0.8804 - val_loss: 0.3448 - val_accuracy: 0.8812
Epoch 3/35
5014/5014 - 20s - loss: 0.3454 - accuracy: 0.8827 - val_loss: 0.3389 - val_accuracy: 0.8850
Epoch 4/35
5014/5014 - 22s - loss: 0.3430 - accuracy: 0.8837 - val_loss: 0.3431 - val_accuracy: 0.8796
Epoch 5/35


KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(testX, testY)
print("Loss:" + str(loss))
print("Accuracy:" + str(accuracy))

In [ ]:
model.summary()

### Conv-LSTM 

In [73]:
X_M.shape

(401119, 12)

In [74]:
X1 = X_M.reshape((-1, 1, 12))

In [75]:
X1.shape

(401119, 1, 12)

In [76]:
(trainX, testX, trainY, testY) = train_test_split(X1, y, test_size = 0.20, random_state = 40)

In [ ]:
nb_filter = 250
filter_length = 3

model = Sequential()
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, padding='same', activation='tanh'))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.2))
model.add(LSTM(17))
model.add(Dropout(0.2))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(trainX, trainY, epochs=35, batch_size=64, verbose=2, validation_data=(testX, testY))

Epoch 1/35
5014/5014 - 29s - loss: 0.3819 - accuracy: 0.7832 - val_loss: 0.3490 - val_accuracy: 0.8001
Epoch 2/35
5014/5014 - 27s - loss: 0.3377 - accuracy: 0.8138 - val_loss: 0.3143 - val_accuracy: 0.8292
Epoch 3/35
5014/5014 - 29s - loss: 0.3231 - accuracy: 0.8231 - val_loss: 0.3048 - val_accuracy: 0.8318
Epoch 4/35
5014/5014 - 27s - loss: 0.3187 - accuracy: 0.8251 - val_loss: 0.3043 - val_accuracy: 0.8319
Epoch 5/35
5014/5014 - 27s - loss: 0.3149 - accuracy: 0.8263 - val_loss: 0.3016 - val_accuracy: 0.8346
Epoch 6/35


In [ ]:
loss, accuracy = model.evaluate(testX, testY)
print("Loss:" + str(loss))
print("Accuracy:" + str(accuracy))

In [ ]:
model.summary()

IF Conv-LSTM doesn't achieve max accuracy then run this, other

In [ ]:
nb_filter = 250
filter_length = 3

model = Sequential()
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, padding='same', activation='tanh'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=nb_filter, kernel_size=filter_length, padding='same', activation='tanh'))
model.add(MaxPooling1D(pool_size=1))
model.add(Activation('softmax'))
model.add(LSTM(17, return_sequences=True))
model.add(LSTM(17, return_sequences=True))
model.add(Dense(128, activation='tanh'))
model.add(Dense(100, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(trainX, trainY, epochs=500, batch_size=100, verbose=2, validation_data=(testX, testY))

In [ ]:
loss, accuracy = model.evaluate(testX, testY)
print("Loss:" + str(loss))
print("Accuracy:" + str(accuracy))

In [ ]:
model.summary()